# Determining Temperature Slopes for Motors

- `f1(t)` = Slope from motor cutoff to max temperature.
- `f2(t)` = Slope from max temperature back to cutoff temperature.

```
Throttle:
   _____________      
 _|             |_______________________
    
Temp:
                  __
                 /  `---.__  
   _____________/          ``---.___
 _/
```

## Test Parameters

- Use 2-4 batteries (max for spec)
- Throttle to 70%
- Back to back runs(2) per day, a second day to check consistency
- Move completed tests to `cooling_slopes`
- File name format `Spike-<MOTOR>-<PWM>_2020_##_##_######.csv`

### Day 1
| Motor   | Day | Filename | Notes |
|---------|-----|----------|-------|
| Mini    | 1/27/2020 | Spike-Mini-1700_2020-01-27_162435.csv | Started hot from motor warm up |
| Mini    | 1/27/2020 | Spike-Mini-1700_2020-01-27_164855.csv |
| P80     | 2/11/2020 | Spike-P80-1700_2020-02-11_162559.csv  | Started hot from motor warm up |
| P80     | 2/11/2020 | Spike-P80-1700_2020-02-11_164450.csv  |
| KDE\*   |
| KDE\*   |
| Mega    | 1/27/2020 | Spike-Mega-1700_2020-01-27_105922.csv | Started hot from motor warm up |
| Mega    | 1/27/2020 | Spike-Mega-1700_2020-01-27_112659.csv | 
| Turnigy |
| Turnigy |

### Day 2
| Motor   | Day | Filename | Notes |
|---------|-----|----------|-------|
| Mini    |
| Mini    |
| P80     |
| P80     |
| KDE\*   |
| KDE\*   |
| Mega    |
| Mega    |
| Turnigy |
| Turnigy |

In [1]:
from analysis_tools import *
import bokeh
from bokeh.plotting import *
output_notebook()

Loading BokehJS ...

In [2]:
unFilteredRuns = runsFromFolder('cooling_slopes')

In [3]:
runs = unFilteredRuns
#runs = list(filter(lambda x: '1200' in x.file, unFilteredRuns))

In [11]:
def mapMotorToColor(run):
    return {
        "Mini": "purple",
        "P80" : "blue",
        "KDE" : "green",
        "Mega": "orange",
        "Turn": "red"
    }[run.details.motor]

In [12]:
plot = figure()

for r in runs:
    plot.line([pt.time for pt in r.data], [pt.motorTemp for pt in r.data], color=mapMotorToColor(r))
    
show(plot)

In [5]:
for run in runs:
    print(run.file)
    spikes = findSpikes(run, [run.details.testPwm])
    
    run.props['cutoffIndex'] = spikes[0]
    run.props['maxTempIndex'] = run.data.index(max(run.data, key=lambda x: x.motorTemp))
    
    print("\t" + str(run.props))

Spike-Mega-1700_2020-01-27_105922.csv
	{'cutoffIndex': 313, 'maxTempIndex': 536}
Spike-Mega-1700_2020-01-27_112659.csv
	{'cutoffIndex': 628, 'maxTempIndex': 834}
Spike-Mini-1700_2020-01-27_162435.csv
	{'cutoffIndex': 473, 'maxTempIndex': 2}
Spike-Mini-1700_2020-01-27_164855.csv
	{'cutoffIndex': 583, 'maxTempIndex': 729}
Spike-P80-1700_2020-02-11_162559.csv
	{'cutoffIndex': 678, 'maxTempIndex': 819}
Spike-P80-1700_2020-02-11_164450.csv
	{'cutoffIndex': 396, 'maxTempIndex': 560}


In [20]:
normalizeTime = True
normalizeTemp = True

plot = figure()
for run in runs:
   
    times = [pt.time for pt in run.data]
    if normalizeTime:
        cutoffTime = run.data[run.props['cutoffIndex']].time
        times = list(map(lambda t: t - cutoffTime, times))
        
    temps = [pt.motorTemp for pt in run.data]
    if normalizeTemp:
        cutoffTemp = run.data[run.props['cutoffIndex']].motorTemp
        temps = list(map(lambda T: T - cutoffTemp, temps))
    
    plot.line(times, temps, color=mapMotorToColor(run))
    
show(plot)